In [2]:
!pip install tensorflow
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.2 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 6.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.6 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 7.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 5.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
import numpy as np
import random

import gym
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete

from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
%load_ext tensorboard

In [2]:
ENV_NAME = 'TRIA-RL-MODEL3D'

In [69]:
TINI= 70.0; HINI= 40.0; AINI= 10.0 

TMIN =-40.0; TMAX=110; HMIN=0.0; HMAX=100.0; AMIN=0.0; AMAX=5000.0

EQUI_LEN= 60

R1= -0.25; R2= -0.5; R3= 2; NR3= -2

CONST_WAIGHT_VEC = [1,1,1,1]

D = {
     0 : [65.0, 80.0, 50.0, 85.0, 40.0, 90.0], 
     1 : [30.0, 50.0, 20.0, 60.0, 10.0, 70.0], 
     2 : [0.0, 19.0, 200.0, 599.0, 600.0, 2000.0]
    }

D1 = { 0: [65.0, 80.0], 1: [30.0, 50.0], 2: [0.0, 20.0]}

In [70]:
class TriaEnv(Env):
    
    def __init__(self):
        self.action_space = MultiDiscrete(np.array([2,2,2,2,2]))
        
        self.observation_space = Box(low=np.array([TMIN, HMIN, AMIN]), high=np.array([TMAX, HMAX, AMAX]))
        
        self.state = [TINI + random.uniform(-1.0,1.0), HINI + random.uniform(-1.0,1.0), AINI + random.uniform(-1.0,1.0)]
        
        #print('^^^', self.state, self.action_space)
        
        self.equilibrium_length = EQUI_LEN
        
    def step(self, action):
        
        ap = [1 if e == 1 else -1 for e in action]
        
        actionPrime = [a*b for a,b in zip(ap, CONST_WAIGHT_VEC)]
        
        actionAlgo = [actionPrime[a] - actionPrime[len(actionPrime) -a -1] for a in range(len(actionPrime) // 2)]
        
        actionAlgo.append(actionPrime[len(actionPrime) // 2])                                                              
        
        #print('***',actionAlgo, self.state)
        
        self.state = [a + b for a, b in zip(actionAlgo, self.state)]
        
        #print('&&&', actionAlgo, self.state)
        
        #reduce tria simulation length by 1 second
        self.equilibrium_length -= 1
        
        reward = [R3 if e >= D[i][0] and e<= D[i][1] else R2 if e >= D[i][2] and e<= D[i][3] else R1 if e >= D[i][4] and e <= D[i][5] else NR3 for i,e in enumerate(self.state)]
        #reward = [R3 if e >= D1[i][0] and e <= D1[i][1] else NR3  for i, e in enumerate(self.state)]

        reward = sum(reward)
        #print('$$$', reward)
            
        if self.equilibrium_length <= 0:
            terminated = True
        else:
            terminated = False
            
        info = {}
        print('reward:{} state:{}'.format(reward, self.state))
        return self.state, reward, terminated, False, {}
    
    def render(self):
        pass
    
    def reset(self):
        
        self.state =[TINI + random.uniform(-1.0,1.0),HINI + random.uniform(-1.0,1.0),AINI + random.uniform(-1.0,1.0)]
        #print('@@@', self.state)
        self.equilibrium_length = EQUI_LEN
        
        return self.state, {}
        

In [71]:
env= TriaEnv()

In [72]:
env.observation_space.sample()

array([  86.87644 ,   25.327301, 3605.3389  ], dtype=float32)

In [73]:
env.action_space.sample()

array([0, 1, 1, 0, 1])

In [74]:
env.state

[70.61256440069322, 40.059430123135776, 10.664474627925387]

In [75]:
#env.reset()

In [76]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    #print(state)
    terminated = False
    score = 0 #[0,0,0] 
    
    while not terminated:
        #env.render()
        action = env.action_space.sample()
        #print(action, terminated , reward)
        #print(env.step(action))
        next_state, reward, terminated, truncated, info = env.step(action) 
        score += reward #[a + b for a, b in zip(reward, score)]
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

reward:6 state:[68.0514402226548, 38.09176381341063, 10.111704741787877]
reward:6 state:[68.0514402226548, 38.09176381341063, 9.111704741787877]
reward:6 state:[68.0514402226548, 36.09176381341063, 10.111704741787877]
reward:6 state:[68.0514402226548, 36.09176381341063, 9.111704741787877]
reward:6 state:[66.0514402226548, 36.09176381341063, 8.111704741787877]
reward:6 state:[68.0514402226548, 36.09176381341063, 7.111704741787877]
reward:6 state:[68.0514402226548, 36.09176381341063, 6.111704741787877]
reward:6 state:[66.0514402226548, 36.09176381341063, 5.111704741787877]
reward:6 state:[66.0514402226548, 38.09176381341063, 4.111704741787877]
reward:6 state:[66.0514402226548, 38.09176381341063, 5.111704741787877]
reward:6 state:[68.0514402226548, 36.09176381341063, 6.111704741787877]
reward:6 state:[68.0514402226548, 36.09176381341063, 7.111704741787877]
reward:6 state:[68.0514402226548, 38.09176381341063, 6.111704741787877]
reward:6 state:[70.0514402226548, 38.09176381341063, 5.1117047

In [77]:
log_path = os.path.join('training','logs')

In [78]:
#from stable_baselines3 import A2C
#from stable_baselines3.common.vec_env import SubprocVecEnv
#env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1) #, tensorboard_log=log_path)
#model = A2C("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.learn(total_timesteps=40000)

reward:6 state:[70.95969990837166, 37.355848682109475, 11.608746418600083]
reward:6 state:[72.95969990837166, 35.355848682109475, 12.608746418600083]
reward:6 state:[70.95969990837166, 33.355848682109475, 13.608746418600083]
reward:6 state:[72.95969990837166, 33.355848682109475, 14.608746418600083]
reward:6 state:[72.95969990837166, 31.355848682109475, 15.608746418600083]
reward:3.5 state:[72.95969990837166, 29.355848682109475, 16.608746418600084]
reward:3.5 state:[72.95969990837166, 29.355848682109475, 15.608746418600084]
reward:3.5 state:[74.95969990837166, 27.355848682109475, 16.608746418600084]
reward:3.5 state:[76.95969990837166, 25.355848682109475, 17.608746418600084]
reward:3.5 state:[76.95969990837166, 25.355848682109475, 16.608746418600084]
reward:3.5 state:[76.95969990837166, 25.355848682109475, 15.608746418600084]
reward:3.5 state:[78.95969990837166, 25.355848682109475, 14.608746418600084]
reward:3.5 state:[76.95969990837166, 23.355848682109475, 15.608746418600084]
reward:3.